In [1]:
import pynamical
from pynamical import simulate, phase_diagram_3d
import pandas as pd, numpy as np, matplotlib.pyplot as plt, random, glob, os, IPython.display as IPdisplay
from PIL import Image
save_folder = 'images/phase-animate'

In [4]:
# run the logistic model and create random noise
chaos_pops = simulate(num_gens=1000, rate_min=3.99, num_rates=1)
random_pops = pd.DataFrame([random.random() for _ in range(0, 1000)], columns=['value'])
pops = pd.concat([chaos_pops, random_pops], axis=1)
pops.columns = ['chaos', 'aletorio']
pops.tail()

,chaos,aletorio
995,0.247214,0.366904
996,0.742535,0.925918
997,0.762795,0.478350
998,0.721947,0.961534
999,0.800951,0.900743


In [8]:
# set a filename and then create the plot
gif_filename = 'Rotando el modelo logístico aleatorio'
working_folder = '{}/{}'.format(save_folder, gif_filename)
if not os.path.exists(working_folder):
    os.makedirs(working_folder)
    
fig, ax = phase_diagram_3d(pops, color=['#003399','#cc0000'], xlabel='Población (t)', ylabel='Población (t + 1)', 
                           zlabel='', legend=True, legend_bbox_to_anchor=(0.94, 0.96), show=False, save=False)

# configure the initial viewing perspective to look straight down at the x-y plane
ax.elev = 40
ax.azim = 180
ax.dist = 10

# sweep the perspective down and rotate to reveal the 3-D structure of the strange attractor
for n in range(0, 100):
    if n >= 20 and n <= 22:
        ax.set_xlabel('')
        ax.set_ylabel('') #don't show axis labels while we move around, it looks weird
        ax.elev = ax.elev-0.5 #start by panning down slowly
    if n >= 23 and n <= 36:
        ax.elev = ax.elev-1.0 #pan down faster
    if n >= 37 and n <= 60:
        ax.elev = ax.elev-1.5
        ax.azim = ax.azim+1.1 #pan down faster and start to rotate
    if n >= 61 and n <= 64:
        ax.elev = ax.elev-1.0
        ax.azim = ax.azim+1.1 #pan down slower and rotate same speed
    if n >= 65 and n <= 73:
        ax.elev = ax.elev-0.5
        ax.azim = ax.azim+1.1 #pan down slowly and rotate same speed
    if n >= 74 and n <= 76:
        ax.elev = ax.elev-0.2
        ax.azim = ax.azim+0.5 #end by panning/rotating slowly to stopping position   
    if n == 77: #add axis labels at the end, when the plot isn't moving around
        ax.set_xlabel('Población (t)')
        ax.set_ylabel('Población (t + 1)')
        ax.set_zlabel('Población (t + 2)')
        
    # add a figure title to each plot then save the figure to the disk
    fig.suptitle(u'Diagrama de fase 3D, Chaos vs aleatorio', fontsize=16, x=0.5, y=0.85)
    plt.savefig('{}/{}/img{:03d}.png'.format(save_folder, gif_filename, n), bbox_inches='tight')

# don't display the static plot
plt.close()

# load all the static images into a list then save as an animated gif
gif_filepath = '{}/{}.gif'.format(save_folder, gif_filename)
images = [Image.open(image) for image in glob.glob('{}/*.png'.format(working_folder))]
gif = images[0]
gif.info['duration'] = 10 #milliseconds per frame
gif.info['loop'] = 0 #how many times to loop (0=infinite)
gif.save(fp=gif_filepath, format='gif', save_all=True, append_images=images[1:])
IPdisplay.Image(url=gif_filepath)